# A Comparison of mannitol and glucose as sole carbon sources

### Setup imports

In [1]:
import cobra
import pandas as pd
import sys

# Disable scientific notation
pd.set_option('display.float_format', '{:.6f}'.format)

source_dir = '../src'
sys.path.append(source_dir)
from add_column_to_13c_flux_df import add_column_to_13c_flux_df
from flux_prediction_scatterplot import flux_prediction_scatterplot

### Load model

In [2]:
model = cobra.io.json.load_json_model("../genome_scale_models/iYLI647_corr_2.json")
model

Name,model
Memory address,11bd7f8b0
Number of metabolites,1121
Number of reactions,1348
Number of genes,648
Number of groups,0
Objective expression,1.0*biomass_C - 1.0*biomass_C_reverse_c1d5c
Compartments,"c, e, m, n, x, r, g, v"


### Add farnesyl diphosphate (FPP) exchange reaction to create a terpenoid reaction to maximize

In [3]:
# create the exchange reaction for farnesyl diphosphate
exchange_reaction = cobra.Reaction('EX_frdp[c]')
exchange_reaction.name = 'Exchange reaction for farnesyl diphosphate'
exchange_reaction.subsystem = 'Exchange'
exchange_reaction.lower_bound = 0
exchange_reaction.upper_bound = 1000
exchange_reaction.add_metabolites({model.metabolites.get_by_id('frdp[c]'): -1})
model.add_reactions([exchange_reaction])

### Add mannitol uptake metabolites and reactions to the model

In [4]:
# Extracellular mannitol
mannitol_e = cobra.Metabolite(
    'mannitol[e]',
    formula='C6H14O6',
    name='Mannitol',
    compartment='C_e')

# Cytosolic mannitol
mannitol_c = cobra.Metabolite(
    'mannitol[c]',
    formula='C6H14O6',
    name='Mannitol',
    compartment='C_c')

mannitol_exchange = cobra.Reaction('EX_mannitol_e')
mannitol_exchange.name = 'mannitol exchange'
mannitol_exchange.lower_bound = 0.  
mannitol_exchange.upper_bound = 1000.  
mannitol_exchange.add_metabolites({
    mannitol_e: -1.0,
})

mannitol_transport = cobra.Reaction('mannitol_transport')
mannitol_transport.name = 'mannitol transport'
mannitol_transport.lower_bound = 0. 
mannitol_transport.upper_bound = 1000. 
mannitol_transport.add_metabolites({
    mannitol_e: -1.0,
    mannitol_c:  1.0,
}) 

# EC 1.14.13.244
# add this reaction: nad_c + mnl_c ⇌ fru_c + h_c + nadh_c
mannitol_dehydrogenase = cobra.Reaction('MNLDH2')
mannitol_dehydrogenase.name = 'mannitol dehydrogenase'
mannitol_dehydrogenase.lower_bound = 0.
mannitol_dehydrogenase.upper_bound = 1000.  # This is the default
mannitol_dehydrogenase.add_metabolites({
    mannitol_c: -1.0,
    model.metabolites.get_by_id('nadp[c]'):   -1.0,
    model.metabolites.get_by_id('h[c]'): 1.0,
    model.metabolites.get_by_id('fru[c]'): 1.0,
    model.metabolites.get_by_id('nadph[c]'):  1.0,
})

model.add_reactions([mannitol_exchange])
model.add_reactions([mannitol_transport])
model.add_reactions([mannitol_dehydrogenase])

### Run biomass-maximizing FBA for glucose

In [5]:
# minimal medium with glycerol as sole carbon source
model.medium = {
  'EX_glc(e)': 100.0, # 100 mmol of glucose
  'EX_h2o(e)': 10000.0,
  'EX_h(e)': 10000.0,
  'EX_nh4(e)': 10000.0,
  'EX_o2(e)': 10000.0,
  'EX_pi(e)': 10000.0,
  'EX_so4(e)': 10000.0,
}
  
with model:
    # update the model to use the specified medium
    medium = model.medium

    # print the medium
    print('Media composition:')
    [print(model.medium[m], m) for m in model.medium]

    # set the objective to be the biomass reaction (this is the default behavior, but we'll be explicit)
    model.objective = model.reactions.get_by_id('EX_biom')

    # find the optimal solution
    solution = cobra.flux_analysis.pfba(model)

    print('\nBiomass flux', solution.fluxes['EX_biom'])

    # make a list of dictionaries with the reaction id, name, flux, and absolute flux
    reactions = []
    for reaction_id, flux in solution.fluxes.items():
      reactions.append({
        'reaction_id': reaction_id,
        'reaction_name': model.reactions.get_by_id(reaction_id).name,
        'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
        'glucose_max_biomass_flux': flux,
        'absolute_flux': abs(flux), # use for sorting, then drop
      })

    # make a dataframe from the list of dictionaries
    fba_results_df = pd.DataFrame(reactions)

    # sort the dataframe by absolute flux
    fba_results_df = fba_results_df.sort_values(by=['absolute_flux'], ascending=False)

    # drop the absolute flux column
    fba_results_df = fba_results_df.drop(columns=['absolute_flux'])

fba_results_df.head()


Media composition:
100.0 EX_glc(e)
10000.0 EX_h2o(e)
10000.0 EX_h(e)
10000.0 EX_nh4(e)
10000.0 EX_o2(e)
10000.0 EX_pi(e)
10000.0 EX_so4(e)

Biomass flux 11.398166174413932


,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-561.830307
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],341.609652
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],335.476108
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,326.533474
421,EX_h2o(e),H2O exchange,h2o[e] <=>,325.265198


### Run FPP-maximizing FBA for glucose

In [6]:
# minimal medium with glucose as sole carbon source
model.medium = {
    'EX_glc(e)': 100.0, # 100 mmol of glucose
    'EX_h2o(e)': 10000.0,
    'EX_h(e)': 10000.0,
    'EX_nh4(e)': 10000.0,
    'EX_o2(e)': 10000.0,
    'EX_pi(e)': 10000.0,
    'EX_so4(e)': 10000.0,
}

solution = 0
  
with model:
    # update the model to use the specified medium
    medium = model.medium

    # print the medium
    print('Media composition:')
    [print(model.medium[m], m) for m in model.medium]

    # set the objective to be the farnesyl diphosphate exchange reaction
    model.objective = model.reactions.get_by_id('EX_frdp[c]')

    # find and display the optimal solution
    solution = cobra.flux_analysis.pfba(model)

    print('\nFPP flux', solution.fluxes['EX_frdp[c]'])

    # create a list to hold the max fpp fluxes
    max_fpp_fluxes = []

    # loop over the reactions in the fba_results_df dataframe, and add a column for FPP maximmizing glucose fluxes
    for _, row in fba_results_df.iterrows():
        reaction_id = row['reaction_id']
        flux = solution.fluxes[reaction_id]
        max_fpp_fluxes.append(flux)

# add the max fpp fluxes to the dataframe
fba_results_df['glucose_max_fpp_flux'] = max_fpp_fluxes

fba_results_df.head()

Media composition:
100.0 EX_glc(e)
10000.0 EX_h2o(e)
10000.0 EX_h(e)
10000.0 EX_nh4(e)
10000.0 EX_o2(e)
10000.0 EX_pi(e)
10000.0 EX_so4(e)

FPP flux 22.351233671988393


,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux,glucose_max_fpp_flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-561.830307,-541.848089
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],341.609652,281.761006
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],335.476108,281.761006
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,326.533474,281.761006
421,EX_h2o(e),H2O exchange,h2o[e] <=>,325.265198,354.136430


### Run biomass-maximizing FBA for mannitol

In [7]:
# minimal medium with glycerol as sole carbon source
model.medium = {
  'EX_mannitol_e': 100.0, # 100 mmol of mannitol
  'EX_h2o(e)': 10000.0,
  'EX_h(e)': 10000.0,
  'EX_nh4(e)': 10000.0,
  'EX_o2(e)': 10000.0,
  'EX_pi(e)': 10000.0,
  'EX_so4(e)': 10000.0,
}
  
with model:
    # update the model to use the specified medium
    medium = model.medium

    # print the medium
    print('Media composition:')
    [print(model.medium[m], m) for m in model.medium]

    # set the objective to be the biomass reaction (this is the default behavior, but we'll be explicit)
    model.objective = model.reactions.get_by_id('EX_biom')

    # find the optimal solution
    solution = cobra.flux_analysis.pfba(model)

    print('\nBiomass flux', solution.fluxes['EX_biom'])
    
    # create a list to hold the max fpp fluxes
    flux_values = []

    # loop over the reactions in the fba_results_df dataframe, and add a column for FPP maximmizing glucose fluxes
    for _, row in fba_results_df.iterrows():
        reaction_id = row['reaction_id']
        flux = solution.fluxes[reaction_id]
        flux_values.append(flux)

# add the max fpp fluxes to the dataframe
fba_results_df['mannitol_max_biomass_flux'] = flux_values

fba_results_df.head()


EX_mannitol_e does not seem to be an an exchange reaction. Applying bounds anyway.


Media composition:
10000.0 EX_h2o(e)
10000.0 EX_h(e)
10000.0 EX_nh4(e)
10000.0 EX_o2(e)
10000.0 EX_pi(e)
10000.0 EX_so4(e)

Biomass flux 12.34801335561521


,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux,glucose_max_fpp_flux,mannitol_max_biomass_flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-561.830307,-541.848089,-607.093558
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],341.609652,281.761006,370.077123
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],335.476108,281.761006,363.432451
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,326.533474,281.761006,353.744597
421,EX_h2o(e),H2O exchange,h2o[e] <=>,325.265198,354.136430,402.370631


### Run FPP-maximizing FBA for mannitol

In [8]:
# minimal medium with glycerol as sole carbon source
model.medium = {
  'EX_mannitol_e': 100.0, # 100 mmol of mannitol
  'EX_h2o(e)': 10000.0,
  'EX_h(e)': 10000.0,
  'EX_nh4(e)': 10000.0,
  'EX_o2(e)': 10000.0,
  'EX_pi(e)': 10000.0,
  'EX_so4(e)': 10000.0,
}

solution = 0
  
with model:
    # update the model to use the specified medium
    medium = model.medium

    # print the medium
    print('Media composition:')
    [print(model.medium[m], m) for m in model.medium]

    # set the objective to be the farnesyl diphosphate exchange reaction
    model.objective = model.reactions.get_by_id('EX_frdp[c]')

    # find and display the optimal solution
    solution = cobra.flux_analysis.pfba(model)

    print('\nFPP flux', solution.fluxes['EX_frdp[c]'])

    # create a list to hold the max fpp fluxes
    max_fpp_fluxes = []

    # loop over the reactions in the fba_results_df dataframe, and add a column for FPP maximmizing mannitol fluxes
    for _, row in fba_results_df.iterrows():
        reaction_id = row['reaction_id']
        flux = solution.fluxes[reaction_id]
        max_fpp_fluxes.append(flux)

# add the max fpp fluxes to the dataframe
fba_results_df['mannitol_max_fpp_flux'] = max_fpp_fluxes

fba_results_df.head()

EX_mannitol_e does not seem to be an an exchange reaction. Applying bounds anyway.


Media composition:
10000.0 EX_h2o(e)
10000.0 EX_h(e)
10000.0 EX_nh4(e)
10000.0 EX_o2(e)
10000.0 EX_pi(e)
10000.0 EX_so4(e)

FPP flux 23.584905660377355


,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux,glucose_max_fpp_flux,mannitol_max_biomass_flux,mannitol_max_fpp_flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-561.830307,-541.848089,-607.093558,-623.270440
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],341.609652,281.761006,370.077123,326.100629
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],335.476108,281.761006,363.432451,326.100629
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,326.533474,281.761006,353.744597,326.100629
421,EX_h2o(e),H2O exchange,h2o[e] <=>,325.265198,354.136430,402.370631,440.566038


### Identify reactions with differential flux values

In [9]:
# split the dataframe into two dataframes, one for biomass and one for fpp
biomass_df = fba_results_df[['reaction_id', 'reaction_name', 'full_reaction', 'glucose_max_biomass_flux', 'mannitol_max_biomass_flux']]
fpp_df = fba_results_df[['reaction_id', 'reaction_name', 'full_reaction', 'glucose_max_fpp_flux', 'mannitol_max_fpp_flux']]

display(biomass_df.head())
display(fpp_df.head())


,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux,mannitol_max_biomass_flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-561.830307,-607.093558
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],341.609652,370.077123
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],335.476108,363.432451
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,326.533474,353.744597
421,EX_h2o(e),H2O exchange,h2o[e] <=>,325.265198,402.370631


,reaction_id,reaction_name,full_reaction,glucose_max_fpp_flux,mannitol_max_fpp_flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-541.848089,-623.270440
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],281.761006,326.100629
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],281.761006,326.100629
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,281.761006,326.100629
421,EX_h2o(e),H2O exchange,h2o[e] <=>,354.136430,440.566038


### Add columns comparing the glucose and mannitol solutions

In [10]:
# a function that adds a columns for difference and percent difference
def add_difference_columns(_df, column_1, column_2):
    df = _df.copy()

    df['difference'] = df[column_1] - df[column_2]
    df['percent_difference'] = df['difference'] / df[column_1] * 100

    # add a column for the absolute value of the percent difference
    df['absolute_difference'] = abs(df['difference'])


    df = df.sort_values(by=['absolute_difference'], ascending=False)

    # drop the absolute percent difference column
    df = df.drop(columns=['absolute_difference'])

    return df

# add the difference columns to the dataframes
biomass_df = add_difference_columns(biomass_df, 'glucose_max_biomass_flux', 'mannitol_max_biomass_flux')
fpp_df = add_difference_columns(fpp_df, 'glucose_max_fpp_flux', 'mannitol_max_fpp_flux')

display(biomass_df.head())
display(fpp_df.head())

,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux,mannitol_max_biomass_flux,difference,percent_difference
360,HEX7,hexokinase D fructoseATP,atp[c] + fru[c] --> adp[c] + f6p[c] + h[c],0.000000,100.000000,-100.000000,-inf
1351,MNLDH2,mannitol dehydrogenase,mannitol[c] + nadp[c] --> fru[c] + h[c] + nadp...,0.000000,100.000000,-100.000000,-inf
413,EX_glc(e),D Glucose exchange,glc_D[e] <=>,-100.000000,0.000000,-100.000000,100.000000
1349,EX_mannitol_e,mannitol exchange,mannitol[e] -->,0.000000,-100.000000,100.000000,inf
1350,mannitol_transport,mannitol transport,mannitol[e] --> mannitol[c],0.000000,100.000000,-100.000000,-inf


,reaction_id,reaction_name,full_reaction,glucose_max_fpp_flux,mannitol_max_fpp_flux,difference,percent_difference
360,HEX7,hexokinase D fructoseATP,atp[c] + fru[c] --> adp[c] + f6p[c] + h[c],0.000000,100.000000,-100.000000,-inf
1351,MNLDH2,mannitol dehydrogenase,mannitol[c] + nadp[c] --> fru[c] + h[c] + nadp...,0.000000,100.000000,-100.000000,-inf
547,PGI,glucose 6 phosphate isomerase,g6p[c] <=> f6p[c],100.000000,-0.000000,100.000000,100.000000
413,EX_glc(e),D Glucose exchange,glc_D[e] <=>,-100.000000,0.000000,-100.000000,100.000000
1349,EX_mannitol_e,mannitol exchange,mannitol[e] -->,0.000000,-100.000000,100.000000,inf


### 

### Glucose vs. mannitol (differential fluxes for max biomass)

In [11]:
# filter the biomass dataframe to only include reactions with greater than 5 flux units for at least one of the carbon sources
biomass_df = biomass_df[(abs(biomass_df['glucose_max_biomass_flux']) > 5) | (abs(biomass_df['mannitol_max_biomass_flux']) > 5)]

biomass_df.head(30)

,reaction_id,reaction_name,full_reaction,glucose_max_biomass_flux,mannitol_max_biomass_flux,difference,percent_difference
360,HEX7,hexokinase D fructoseATP,atp[c] + fru[c] --> adp[c] + f6p[c] + h[c],0.000000,100.000000,-100.000000,-inf
1351,MNLDH2,mannitol dehydrogenase,mannitol[c] + nadp[c] --> fru[c] + h[c] + nadp...,0.000000,100.000000,-100.000000,-inf
413,EX_glc(e),D Glucose exchange,glc_D[e] <=>,-100.000000,0.000000,-100.000000,100.000000
1349,EX_mannitol_e,mannitol exchange,mannitol[e] -->,0.000000,-100.000000,100.000000,inf
1350,mannitol_transport,mannitol transport,mannitol[e] --> mannitol[c],0.000000,100.000000,-100.000000,-inf
958,GLCt1,glucose transport uniport,glc_D[e] --> glc_D[c],100.000000,0.000000,100.000000,100.000000
711,HEX1,hexokinase D glucoseATP,atp[c] + glc_D[c] --> adp[c] + g6p[c] + h[c],100.000000,0.000000,100.000000,100.000000
421,EX_h2o(e),H2O exchange,h2o[e] <=>,325.265198,402.370631,-77.105433,-23.705405
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-325.265198,-402.370631,77.105433,-23.705405
547,PGI,glucose 6 phosphate isomerase,g6p[c] <=> f6p[c],35.574485,-19.794308,55.368793,155.641870


### Glucose vs. mannitol (differential fluxes for max FPP)

In [12]:
# filter the fpp dataframe to only include reactions with greater than 5 flux units for at least one of the carbon sources
fpp_df = fpp_df[(abs(fpp_df['glucose_max_fpp_flux']) > 5) | (abs(fpp_df['mannitol_max_fpp_flux']) > 5)]

fpp_df.head(30)

,reaction_id,reaction_name,full_reaction,glucose_max_fpp_flux,mannitol_max_fpp_flux,difference,percent_difference
360,HEX7,hexokinase D fructoseATP,atp[c] + fru[c] --> adp[c] + f6p[c] + h[c],0.000000,100.000000,-100.000000,-inf
1351,MNLDH2,mannitol dehydrogenase,mannitol[c] + nadp[c] --> fru[c] + h[c] + nadp...,0.000000,100.000000,-100.000000,-inf
547,PGI,glucose 6 phosphate isomerase,g6p[c] <=> f6p[c],100.000000,-0.000000,100.000000,100.000000
413,EX_glc(e),D Glucose exchange,glc_D[e] <=>,-100.000000,0.000000,-100.000000,100.000000
1349,EX_mannitol_e,mannitol exchange,mannitol[e] -->,0.000000,-100.000000,100.000000,inf
1350,mannitol_transport,mannitol transport,mannitol[e] --> mannitol[c],0.000000,100.000000,-100.000000,-inf
958,GLCt1,glucose transport uniport,glc_D[e] --> glc_D[c],100.000000,0.000000,100.000000,100.000000
711,HEX1,hexokinase D glucoseATP,atp[c] + glc_D[c] --> adp[c] + g6p[c] + h[c],100.000000,-0.000000,100.000000,100.000000
421,EX_h2o(e),H2O exchange,h2o[e] <=>,354.136430,440.566038,-86.429608,-24.405738
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-354.136430,-440.566038,86.429608,-24.405738


### Print out details of mevalonate pathway
The purpose of this was to determine the reason why carotenoids require NADPH.
<br><br>
The answer was the reaction of HMG-CoA + 2 NADPH -> Mevalonate (HMG-CoA reductase)

In [13]:
def print_metablite_details(metabolite_id):
    metabolite = model.metabolites.get_by_id(metabolite_id)
    print(f'{metabolite.name} ({metabolite.id}) reactions:')
    for reaction in metabolite.reactions:
        print(reaction, reaction.name)

    print()

print_metablite_details('accoa[c]')
print_metablite_details('hmgcoa[c]')
print_metablite_details('mev_R[c]')
print_metablite_details('5pmev[c]')
print_metablite_details('5dpmev[c]')
print_metablite_details('ipdp[c]')
print_metablite_details('dmpp[c]')
print_metablite_details('grdp[c]')
print_metablite_details('frdp[c]')

Acetyl_CoA (accoa[c]) reactions:
ACACCT: acac[c] + accoa[c] --> aacoa[c] + ac[c] Acetyl-CoA:acetoacetyl-CoA transferase
IPPS: 3mob[c] + accoa[c] + h2o[c] --> 3c3hmp[c] + coa[c] + h[c] 2 isopropylmalate synthase
SPMDAT1: accoa[c] + spmd[c] --> N1aspmd[c] + coa[c] + h[c] Spermidine acetyltransferase
ACCOACr: accoa[c] + atp[c] + hco3[c] <=> adp[c] + h[c] + malcoa[c] + pi[c] acetyl CoA carboxylase reversible reaction
FAS80_L: accoa[c] + 8.0 h[c] + 3.0 malcoa[c] + 6.0 nadph[c] --> 3.0 co2[c] + 4.0 coa[c] + 2.0 h2o[c] + 6.0 nadp[c] + octa[c] fatty acid synthase n C80 lumped reaction
ATPCitL: atp[c] + cit[c] + coa[c] --> accoa[c] + adp[c] + oaa[c] + pi[c] ATP Citrate Lyase YL
ACGAM6PS: accoa[c] + gam6p[c] <=> acgam6p[c] + coa[c] + h[c] N acetylglucosamine 6 phosphate synthase
CSNAT: accoa[c] + crn[c] --> acrn[c] + coa[c] carnitine O acetyltransferase
HSERTA: accoa[c] + hom_L[c] <=> achms[c] + coa[c] homoserine O trans acetylase
ACACT1: 2.0 accoa[c] <=> aacoa[c] + coa[c] acetyl CoA C acetyltra